In [5]:
import numpy as np

In [6]:
def inviscidFluxes(rho,rhou,rhov,rhow,rhoE):
    
    ## return the inviscid fluxes
    
    return

In [34]:
def eigenDecomp1D(rho,rhou,a,gam):
    
    ## return the eigendecomposition of the flux jacobian (1-D)
    
    u = rhou/rho
    gm1 = gam - 1.
    
    lam = np.zeros(3)
    lam[0] = u - a
    lam[1] = u
    lam[2] = u + a
    
    rightEV = np.zeros( (3,3) )
    leftEV  = np.zeros( (3,3) )
    
    rightEV[:,0] = ( 1., u-a, a**2/gm1 + .5*u**2 - u*a )
    rightEV[:,1] = ( 1., u, .5*u**2 )
    rightEV[:,2] = ( 1., u+a, a**2/gm1 + .5*u**2 + u*a )
    
    leftEV[0,:] = ( gm1/4.*u**2/a**2 + u/(2.*a), -gm1/a**3*u - 1./(2*a), gm1/(2.*a**2) )
    leftEV[1,:] = ( 1. - gm1/2.*u**2/a**2, gm1*u/a**2, -gm1/a**2 )
    leftEV[2,:] = ( gm1/4.*u**2/a**2 - u/(2.*a), -gm1/a**3*u + 1./(2*a), gm1/(2.*a**2) )
    
    return lam, leftEV, rightEV

In [20]:
def eigenDecomp2D(rho,rhou,rhov,a,gam,nx,ny):
    
    ## return the eigendecomposition of the flux jacobian (2-D)
    
    u = rhou/rho
    v = rhov/rho
    gm1 = gam - 1.
    vn = u*nx + v*ny
    
    ke = .5*(u**2 + v**2)
    
    lam = np.zeros(4)
    
    lam[0] = vn - a
    lam[1] = vn
    lam[2] = vn + a
    lam[3] = vn
    
    rightEV = np.zeros( (4,4) )
    leftEV  = np.zeros( (4,4) )
    
    rightEV[:,0] = ( 1., u - a*nx, v - a*ny, a**2/gm1 + ke - vn*a )
    rightEV[:,1] = ( 1., u, v, ke )
    rightEV[:,2] = ( 1., u + a*nx, v + a*ny, a**2/gm1 + ke + vn*a )
    rightEV[:,3] = ( 0., ny, -nx, u*ny - v*nx )
    
    leftEV[0,:] = 1./(2.*a**2)*np.array( [gm1*ke + a*vn, -gm1*u - a*nx, -gm1*v - a*ny, gm1] )
    leftEV[1,:] = 1./a**2*np.array( [a**2 - gm1*ke, gm1*u, gm1*v, -gm1] )
    leftEV[2,:] = 1./(2.*a**2)*np.array( [gm1*ke - a*vn, -gm1*u + a*nx, -gm1*v + a*ny, gm1] )
    leftEV[3,:] = ( v*nx - u*ny, ny, -nx, 0. )
    
    return lam, leftEV, rightEV
    

In [53]:
def eigenDecomp3D(rho,rhou,rhov,rhow,a,gam,nx,ny,nz):
    
    ## return the eigendecomposition of the flux jacobian (3-D)
    
    u = rhou/rho
    v = rhov/rho
    w = rhow/rho
    gm1 = gam - 1.
    vn = u*nx + v*ny + w*nz 
    
    ke = .5*(u**2 + v**2 + w**2)
    
    lam = np.zeros(5)
    
    lam[0] = vn - a
    lam[1] = vn
    lam[2] = vn + a
    lam[3] = vn
    lam[4] = vn
    
    maxN = np.argmax( np.abs( np.array([nx,ny,nz] ) ) )
    
    rightEV = np.zeros( (5,5) )
    leftEV  = np.zeros( (5,5) )
    
    rightEV[:,0] = ( 1., u - a*nx, v - a*ny, w - a*nz, a**2/gm1 + ke - vn*a )
    rightEV[:,1] = ( 1., u, v, w, ke )
    rightEV[:,2] = ( 1., u + a*nx, v + a*ny, w + a*nz, a**2/gm1 + ke + vn*a )
    
    leftEV[0,:] = 1./(2.*a**2)*np.array( [gm1*ke + a*vn, -gm1*u - a*nx, -gm1*v - a*ny, -gm1*w - a*nz, gm1] )
    leftEV[1,:] = 1./a**2*np.array( [a**2 - gm1*ke, gm1*u, gm1*v, gm1*w, -gm1 ] )
    leftEV[2,:] = 1./(2.*a**2)*np.array( [ gm1*ke - a*vn, -gm1*u + a*nx, -gm1*v + a*ny, -gm1*w + a*nz, gm1 ] )
    
    if maxN == 0:
        ## nx biggest
        rightEV[:,3] = ( 0., ny, -nx, 0., u*ny - v*nx )
        rightEV[:,4] = ( 0., -nz, 0., nx, w*nx - u*nz )
        
        leftEV[3,:] = ( (v - vn*ny)/nx, ny, (ny**2-1.)/nx, ny*nz/nx, 0. )
        leftEV[4,:] = ( (vn*nz - w)/nx, -nz, -ny*nz/nx, (1.-nz**2)/nx, 0. )
        
    elif maxN == 1:
        ## ny biggest
        rightEV[:,3] = ( 0., ny, -nx, 0., u*ny - v*nx )
        rightEV[:,4] = ( 0., 0., nz, -ny, v*nz - w*ny )
        
        leftEV[3,:] = ( (vn*nx - u)/ny, (1.-nx**2)/ny, -nx, -nx*nz/ny, 0. )
        leftEV[4,:] = ( (w - vn*nz)/ny, nx*nz/ny, nz, (nz**2 - 1.)/ny, 0. )
        
    else:
        ## nz biggest
        rightEV[:,3] = ( 0., -nz, 0., nx, w*nx - u*nz )
        rightEV[:,4] = ( 0., 0., nz, -ny, v*nz - w*ny )
        
        leftEV[3,:] = ( (u-vn*nx)/nz, (nx**2 - 1.)/nz, nx*ny/nz, nx, 0. )
        leftEV[4,:] = ( (vn*ny - v)/nz, -nx*ny/nz, (1. - ny**2)/nz, -ny, 0. )
        
    return lam, leftEV, rightEV

In [54]:
def generateInputOutput(ndim=1):
    
    rng = np.random.default_rng()
    
    rho = rng.uniform(1.,10.)
    u = rng.uniform(-10.,10.)
    v = rng.uniform(-10.,10.)
    w = rng.uniform(-10.,10.)
    nx = 1.
    a = 5. ## TODO realistic values?
    gamma = 1.4
    
    rhou = rho*u
    rhov = rho*v
    rhow = rho*w

    if ndim == 1:
        lam,LEV,REV = eigenDecomp1D(rho,rhou,a,gamma)
    elif ndim == 2:
        nx = rng.uniform(-1.,1.)
        nyMag = np.sqrt(1. - nx**2)
        ny = nyMag * rng.choice([1.,-1.])
        lam,LEV,REV = eigenDecomp2D(rho,rhou,rhov,a,gamma,nx,ny)
    elif ndim == 3:
        nx = rng.uniform(-1.,1.)
        nyMag = rng.uniform(0.,np.sqrt(1. - nx**2))
        ny = nyMag * rng.choice([1.,-1.])
        nzMag = np.sqrt(1. - nx**2 - ny**2)
        nz = nzMag * rng.choice([1.,-1.])
        lam,LEV,REV = eigenDecomp3D(rho,rhou,rhov,rhow,a,gamma,nx,ny,nz)
    
    
    ## Print out format so we can plop directly in code...
    
    print( f"AD rhoux = {rhou}; AD rho = {rho}; AD a = {a}; ScalarT gam = {gamma};\n" )
    
    if ndim == 2:
        print ( f"AD rhouy = {rhov}; ScalarT nx = {nx}; ScalarT ny = {ny};\n" )
    if ndim == 3:
        print ( f"AD rhouy = {rhov}; AD rhouz = {rhow}; ScalarT nx = {nx}; ScalarT ny = {ny}; ScalarT nz = {nz};\n" )
    
    lamstring = ""
    LEVstring = ""
    REVstring = ""
    
    for j in range(2+ndim):
        lamstring += f"lamExact({j}) = {lam[j]}; "
        for i in range(2+ndim):
            LEVstring += f"LEvExact({i},{j}) = {LEV[i,j]}; "
            REVstring += f"REvExact({i},{j}) = {REV[i,j]}; "
        LEVstring += "\n"
        REVstring += "\n"
    lamstring += "\n"
    
    print (lamstring)
    print (LEVstring)
    print (REVstring)

In [57]:
generateInputOutput(3)

AD rhoux = -52.48911557656745; AD rho = 8.973275634066631; AD a = 5.0; ScalarT gam = 1.4;

AD rhouy = -49.07363972214395; AD rhouz = -64.02380041979175; ScalarT nx = -0.6435391026342729; ScalarT ny = -0.05138386262247015; ScalarT nz = -0.7636865338885254;

lamExact(0) = 4.4942481654751045; lamExact(1) = 9.494248165475105; lamExact(2) = 14.494248165475105; lamExact(3) = 9.494248165475105; lamExact(4) = 9.494248165475105; 

LEvExact(0,0) = 1.409554657866186; LEvExact(1,0) = 0.07974031736264905; LEvExact(2,0) = -0.489294975228835; LEvExact(3,0) = -0.34101250559584656; LEvExact(4,0) = -6.522328702983507; 
LEvExact(0,1) = 0.11114985655291375; LEvExact(1,1) = -0.09359189257897291; LEvExact(2,1) = -0.01755796397394085; LEvExact(3,1) = 0.7671438442126463; LEvExact(4,1) = 0.043299866338580795; 
LEvExact(0,2) = 0.048889312210259224; LEvExact(1,2) = -0.08750185189602443; LEvExact(2,2) = 0.0386125396857652; LEvExact(3,2) = -0.043299866338580795; LEvExact(4,2) = -1.3059804702639668; 
LEvExact(0,3) 